<a href="https://colab.research.google.com/github/Netdrum/MARIA/blob/main/HF_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install huggingface_hub

In [ ]:
from datasets import load_dataset
import datasets
import json
import os

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Define the dataset name and path
dataset_name = "IRCG_VHF"  # Replace with your desired dataset name
dataset_path = "/content/drive/MyDrive/Dataset"  # Replace with the path to your dataset

def load_dataset(dataset_path):
  """Loads the dataset from the specified path.

  Args:
    dataset_path: Path to the dataset directory.

  Returns:
    A Hugging Face dataset.
  """

In [ ]:
# in this cell added in code to split the dataset into train and test splite and push to hf

from datasets import load_dataset, Audio, Features, Value
import json
import os

# Define the data directory
data_dir = "/content/drive/MyDrive/Dataset"

# Define the dataset features
features = Features({
    "file_path": Value("string"),
    "duration": Value("float32"),
    "sample_rate": Value("int32"),
    "length": Value("int32"),
    "frame_rate": Value("int32"),
    "audio": Audio(sampling_rate=16000),
    "transcription": Value("string")
})

# Load the dataset, extracting metadata from JSON files
def extract_metadata(example):
    # Assuming JSON files are named similarly to audio files with a .json extension:
    json_file_path = example["file_path"]  # Get the JSON file path

    # Prepend the data directory to the JSON file path
    json_file_path = os.path.join(data_dir, json_file_path)

    # Add .json extension if it's missing
    if not json_file_path.lower().endswith(".json"):
        json_file_path += ".json"

    # Check if the JSON file exists
    if not os.path.exists(json_file_path):
        print(f"Warning: JSON file not found: {json_file_path}")
        return example  # Skip this example if JSON file is not found

    try:
        # Open the JSON file with error handling and encoding specified
        with open(json_file_path, 'r', encoding='latin-1') as f:
            metadata = json.load(f)
            example["duration"] = metadata["duration"]
            example["sample_rate"] = metadata["sample_rate"]
            example["length"] = metadata["length"]
            example["frame_rate"] = metadata["frame_rate"]
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file: {json_file_path}")
        print(f"Error message: {e}")
        return example  # Skip this example if JSON decoding fails

    # Construct the audio file path based on the JSON file path
    audio_file_path = json_file_path.replace(".json", ".wav")  # Change .json to .wav

    # Update the "audio" field with the path to the audio file
    example["audio"] = {"path": audio_file_path}

    return example

dataset = load_dataset("json", data_files={"train": f"{data_dir}/*.json"}, features=features)
dataset = dataset.map(extract_metadata)

# Split the dataset into train and test
train_testvalid_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

# Extract train and test datasets
train_dataset = train_testvalid_dataset["train"]
test_dataset = train_testvalid_dataset["test"]

# Push the datasets to the Hugging Face Hub
train_dataset.push_to_hub("IRCG_VHF", split="train")
test_dataset.push_to_hub("IRCG_VHF", split="test")